In [1]:
import psycopg2
import pandas as pd
from io import StringIO
import csv

In [ ]:
def db_conn():
    try:
        conn=psycopg2.connect(
            host=input('Enter host name: '),
            port=eval(input('Enter port: ')),
            database=input('Ente database name: '),
            user=input('Enter username: '),
            password=input('Enter password: ')
            )
        print('Connected to database successfully...')
    except Exception as e:
        return f'{e}'
db_conn()

In [19]:
conn.autocommit=True

In [25]:
def crt_schema():
    try:
        cur=conn.cursor()
        cur.execute(f"""CREATE SCHEMA IF NOT EXISTS private;""")
        print('Schema created...')
    except Exception as e:
        return f'{e}'
crt_schema()

Schema created...


In [47]:
def crt_table():
    try:
        cur=conn.cursor()
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS master_data (
                id SERIAL PRIMARY KEY,
                patient_first_name VARCHAR(100),
                patient_last_name VARCHAR(100),
                patient_dob DATE,
                patient_ssn VARCHAR(20),
                patient_phone VARCHAR(30),
                patient_email VARCHAR(255),
                patient_credit_card VARCHAR(30),
                patient_credit_card_expiry VARCHAR(10),
                patient_credit_card_cvv VARCHAR(6),
                insurance_provider VARCHAR(100),
                insurance_id VARCHAR(50),
                age INTEGER,
                admission_date DATE,
                release_date DATE,
                primary_diagnosis TEXT,
                icd_10_code VARCHAR(10),
                diagnosis_category VARCHAR(100),
                attending_physician VARCHAR(150),
                died BOOLEAN DEFAULT FALSE,
                cause_of_death TEXT,
                death_date DATE NULL,
                hospital_name VARCHAR(200),
                transferred BOOLEAN DEFAULT FALSE,
                transfer_hospitals TEXT,
                transfer_date DATE,
                follow_up_appointment DATE NULL,
                follow_up_doctor VARCHAR(150),
                total_cost DECIMAL(10,2),
                insurance_coverage DECIMAL(10,2),
                out_of_pocket DECIMAL(10,2),
                patient_id VARCHAR(50),
                encounter_id VARCHAR(50) NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        print('Table created...')
    except Exception as e:
        return f'{e}'
crt_table()

Table created...


In [ ]:
def load_data():
    try:
        cur = conn.cursor()
        file_path = '/test_data.csv'
        # Read CSV file
        print(f"Reading CSV file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Found {len(df)} records")
        # Get the exact column names from your CSV
        print(f"CSV columns: {list(df.columns)}")
        # Prepare data - convert NaN to None (NULL in SQL)
        df = df.where(pd.notnull(df), None)
        # For StringIO method - need to explicitly list columns in the same order as CSV
        # OPTION 1: Using StringIO (if columns match exactly)
        output = StringIO()
        df.to_csv(output, sep=',', header=False, index=False, quoting=csv.QUOTE_MINIMAL)
        output.seek(0)
        # Define columns explicitly in the order they appear in CSV
        csv_columns = list(df.columns)
        print(f"Loading {len(df.columns)} columns: {csv_columns}")
        # Use copy_from with StringIO
        cur.copy_from(
            output,
            'master_data',
            sep=',',
            null='',
            columns=csv_columns
        )
        conn.commit()
        print(f"Successfully loaded {len(df)} records using COPY")
        
    except Exception as e:
        print(f"Error loading data: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()
load_data()